<a href="https://colab.research.google.com/github/ayeshaaliasghar/medical-policy-rag/blob/main/assignment_3/task1_medical/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain langchain-community sentence-transformers faiss-cpu jsonlines gradio transformers accelerate
print("All assignment packages installed.")


All assignment packages installed.


In [3]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", uploaded.keys())

Saving mtsamples.csv to mtsamples (3).csv
Uploaded: dict_keys(['mtsamples (3).csv'])


In [4]:
import pandas as pd
import os, jsonlines

os.makedirs("data/processed", exist_ok=True)

csv_path = "mtsamples.csv"
df = pd.read_csv(csv_path)

chunks = []

def chunk_text(text, chunk_size=800, overlap=150):
    words = str(text).split()
    if len(words) <= chunk_size:
        return [" ".join(words)]
    out=[]
    step = chunk_size-overlap
    i=0
    while i < len(words):
        out.append(" ".join(words[i:i+chunk_size]))
        i += step
    return out

for idx, row in df.iterrows():
    text = str(row.get("transcription","")).strip()
    if not text: continue
    parts = chunk_text(text)
    for cid, chunk in enumerate(parts):
        chunks.append({
            "text": chunk,
            "metadata": {
                "source": f"mtsample_{idx}",
                "chunk_id": cid
            }
        })

outpath = "data/processed/medical_chunks.jsonl"
with jsonlines.open(outpath, "w") as w:
    w.write_all(chunks)

print(f"Created {len(chunks)} chunks → {outpath}")


Created 5763 chunks → data/processed/medical_chunks.jsonl


In [6]:
import jsonlines
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
import numpy as np
import os

os.makedirs("medical_faiss", exist_ok=True)

# Load chunks
items = []
with jsonlines.open("data/processed/medical_chunks.jsonl") as f:
    for obj in f:
        items.append(obj)

texts = [i["text"] for i in items]
metas = [i["metadata"] for i in items]

print("Loaded chunks:", len(texts))

# ========== FIX: Create LangChain Embeddings Wrapper ==========
class STEmbeddings(Embeddings):
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")

    def embed_documents(self, docs):
        return self.model.encode(docs, convert_to_numpy=True).tolist()

    def embed_query(self, query):
        return self.model.encode([query], convert_to_numpy=True)[0].tolist()

emb = STEmbeddings()

# Build documents
docs = [Document(page_content=t, metadata=m) for t, m in zip(texts, metas)]

# Build FAISS index
db = FAISS.from_texts(
    texts=[d.page_content for d in docs],
    embedding=emb,
    metadatas=[d.metadata for d in docs]
)

db.save_local("medical_faiss")
print("FAISS index successfully saved → 'medical_faiss/'")


Loaded chunks: 5763
FAISS index successfully saved → 'medical_faiss/'


In [8]:
from langchain_community.vectorstores import FAISS

emb = STEmbeddings()  # same embedding class

# Load FAISS components manually
import pickle, faiss, os

faiss_path = "medical_faiss/index.faiss"
pkl_path = "medical_faiss/index.pkl"

# Load FAISS index
index = faiss.read_index(faiss_path)

# Load metadata/docstore
with open(pkl_path, "rb") as f:
    docstore, index_to_docstore_id = pickle.load(f)

# Reconstruct FAISS object properly
db = FAISS(
    embedding_function=emb,
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

retriever = db.as_retriever(search_kwargs={"k": 4})

print("Retriever loaded successfully!")


Retriever loaded successfully!


In [9]:
from transformers import pipeline
from langchain.llms.base import LLM
from typing import Optional, List

pipe = pipeline("text2text-generation", model="google/flan-t5-base")

class FlanLLM(LLM):
    def _call(self, prompt: str, stop=None):
        out = pipe(prompt, max_length=200)[0]["generated_text"]
        return out

    @property
    def _identifying_params(self):
        return {"name": "flan_t5_base"}

    @property
    def _llm_type(self):
        return "custom"

llm = FlanLLM()
print("FLAN-T5 LLM ready!")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


FLAN-T5 LLM ready!


In [10]:
def ask_medical(question):
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([f"[{d.metadata}] {d.page_content}" for d in docs])

    prompt = f"""
You are a medical assistant. Use ONLY the context to answer.

CONTEXT:
{context}

QUESTION: {question}

ANSWER:
"""
    response = llm(prompt)
    return response

print(ask_medical("What are symptoms of sinusitis?"))


/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (4257 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


postnasal drainage, sore throat, facial pain, coughing, headaches and congestion. Additional symptoms include snoring, nasal burning and teeth pain. The symptoms are characterized as moderate to severe. Symptoms are worse in the evening and morning.


In [11]:
import gradio as gr

def query_fn(q):
    return ask_medical(q)

demo = gr.Interface(fn=query_fn, inputs="text", outputs="text", title="Medical RAG QA System")
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://556e33d082ef9f6af4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
evaluation_questions = [
    "What are the symptoms of sinusitis?",
    "What are common symptoms of appendicitis?",
    "What symptoms indicate a possible stroke?",
    "What are the signs of a urinary tract infection?",
    "What are symptoms of pneumonia?",
    "What are symptoms of chronic migraine?",
    "How does a patient typically present with gallstones?",
    "What are symptoms of congestive heart failure?",
    "What causes kidney stones?",
    "What causes chronic back pain?",
    "What causes shortness of breath?",
    "What causes abdominal bloating?",
    "How is appendicitis diagnosed?",
    "How is pneumonia diagnosed?",
    "What tests are used to diagnose diabetes?",
    "How is heart failure confirmed?",
    "What is the treatment for sinus infections?",
    "How is pneumonia treated?",
    "What is the treatment for kidney stones?",
    "What treatments are available for chronic migraines?",
    "How is congestive heart failure managed?",
    "What happens during an abdominal ultrasound?",
    "How is a colonoscopy performed?",
    "What is the procedure for hip replacement surgery?",
    "How is laparoscopic cholecystectomy performed?",
    "What medications are commonly used for hypertension?",
    "What medications treat acid reflux?",
    "What antibiotics are used for urinary tract infections?",
    "What are the risk factors for heart disease?",
    "What lifestyle changes help manage diabetes?"
]

len(evaluation_questions)


30

In [14]:
results = []

for q in evaluation_questions:
    ans = ask_medical(q)
    results.append({"question": q, "answer": ans})
    print("Q:", q)
    print("A:", ans)
    print("--------------------------------------------------")


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What are the symptoms of sinusitis?
A: postnasal drainage, sore throat, facial pain, coughing, headaches and congestion. Additional symptoms include snoring, nasal burning and teeth pain. The symptoms are characterized as moderate to severe. Symptoms are worse in the evening and morning.
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What are common symptoms of appendicitis?
A: nausea and fever
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Q: What medications treat acid reflux?
A: Imitrex
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What antibiotics are used for urinary tract infections?
A: Detrol, Ditropan, Sanctura, and VESIcare
--------------------------------------------------


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What are the risk factors for heart disease?
A: History of hypertension, no history of diabetes mellitus, nonsmoker, cholesterol status unclear, no prior history of coronary artery disease, and heart murmur., active smoker. Cholesterol status, borderline elevated. No history of established coronary artery disease. Family history positive., FAMILY HISTORY: , His father died of coronary artery disease., active smoker.
--------------------------------------------------
Q: What lifestyle changes help manage diabetes?
A: Weight reduction
--------------------------------------------------


In [15]:
import jsonlines
import os

os.makedirs("data/processed", exist_ok=True)

with jsonlines.open("data/processed/evaluation.jsonl", "w") as writer:
    writer.write_all(results)

print("Saved evaluation to data/processed/evaluation.jsonl")


Saved evaluation to data/processed/evaluation.jsonl
